# Managing Portfolio Personal Project Calculation

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import math

In [6]:
# 1. Tickers, shares, and benchmark
tickers = ['VNQ', 'HYG', 'TIP', 'GOVT', 'LQD', 'IJH', 'SPY', 'VWO', 'EFA']
shares = {
    'VNQ': 21, 'HYG': 25, 'TIP': 18, 'GOVT': 260, 'LQD': 36,
    'IJH': 65, 'SPY': 16, 'VWO': 89, 'EFA': 74
}
benchmark_ticker = '^GSPC'

# 2. Date range
start_date = (pd.Timestamp.today() - pd.tseries.offsets.BDay(45)).strftime('%Y-%m-%d')
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')

print(f"Downloading data from {start_date} to {end_date}...")

# 3. Download data
data = yf.download(tickers, start=start_date, end=end_date)['Close']
benchmark_data = yf.download(benchmark_ticker, start=start_date, end=end_date)['Close']

# 4. Daily returns
daily_returns = data.pct_change().dropna()
benchmark_returns = benchmark_data.pct_change().dropna()

# 5. Portfolio value calculation
portfolio_values = pd.DataFrame({ticker: data[ticker] * shares[ticker] for ticker in tickers})
total_value = portfolio_values.sum(axis=1)

# 6. Weighting and portfolio return
weights = portfolio_values.div(total_value, axis=0)
portfolio_daily_return = (daily_returns * weights).sum(axis=1)

# 7. Combine with benchmark
combined_df = pd.concat([portfolio_daily_return, benchmark_returns], axis=1).dropna()
combined_df.columns = ['Portfolio', 'SP500']

# 8. Performance metrics 
start_value = total_value.iloc[0]
end_value = total_value.iloc[-1]
years = 1

# Annuanlized_ return 
annualized_return = (end_value / start_value) ** (1 / years) - 1

# Other metrics
avg_daily_return = combined_df['Portfolio'].mean()
daily_volatility = combined_df['Portfolio'].std()
annualized_volatility = daily_volatility * np.sqrt(252)

risk_free_rate = 0.02
sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility

# Information Ratio
excess_returns = combined_df['Portfolio'] - combined_df['SP500']
mean_excess_return = excess_returns.mean() * 252
tracking_error = excess_returns.std() * np.sqrt(252)
information_ratio = mean_excess_return / tracking_error if tracking_error != 0 else np.nan

# Treynor Ratio
cov_matrix = np.cov(combined_df['Portfolio'], combined_df['SP500'])
beta = cov_matrix[0, 1] / np.var(combined_df['SP500']) if np.var(combined_df['SP500']) != 0 else np.nan
treynor_ratio = (annualized_return - risk_free_rate) / beta if beta and beta != 0 else np.nan

# 9. Output
print("\n==== Portfolio Performance (CAGR-based) ====\n")
print(f"Start Value: ${start_value:,.2f}")
print(f"End Value: ${end_value:,.2f}")
print(f"Years: {years:.4f}")
print(f"Annualized Return: {annualized_return:.4%}")
print(f"Annualized Volatility: {annualized_volatility:.4%}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
print(f"Information Ratio: {information_ratio:.4f}")
print(f"Beta: {beta:.4f}")
print(f"Treynor Ratio: {treynor_ratio:.4f}")

[*********************100%***********************]  9 of 9 completed


[*********************100%***********************]  1 of 1 completed


==== Portfolio Performance (CAGR-based) ====

Start Value: $39,261.84
End Value: $37,049.64
Years: 1.0000
Annualized Return: -5.6345%
Annualized Volatility: 21.1944%
Sharpe Ratio: -0.3602
Information Ratio: 2.3485
Beta: 0.5925
Treynor Ratio: -0.1288


# Portfolio Performance

In [7]:

# Store all metrics in a dictionary
portfolio_stats = {
    "Start Value": f"${start_value:,.2f}",
    "End Value"  :f"${end_value:,.2f}",
    "Average Daily Return": avg_daily_return,
    "Average Daily Std Dev (Volatility)": daily_volatility,
    "Average Annualized Return": annualized_return,
    "Average Annualized Std Dev": annualized_volatility,
    "Sharpe Ratio": sharpe_ratio,
    "Information Ratio": information_ratio,
    "Treynor Ratio": treynor_ratio,
    "Beta (vs SP500)": beta
}
Portfolio = pd.DataFrame(portfolio_stats,index=["Portfolio Stats"])
Portfolio = Portfolio.transpose()
Portfolio

,Portfolio Stats
Start Value,"$39,261.84"
End Value,"$37,049.64"
Average Daily Return,-0.001145
Average Daily Std Dev (Volatility),0.013351
Average Annualized Return,-0.056345
Average Annualized Std Dev,0.211944
Sharpe Ratio,-0.360211
Information Ratio,2.348455
Treynor Ratio,-0.128848
Beta (vs SP500),0.59252
